In [50]:
# Check these before running
# KPUM Workstation
sourcedatabasefolder = '/mnt/d/Finn/KPUM_NODDI/DICOM_KPUM_NODDI/sourcedata'
studydir = ''
# Finn's Laptop
sourcedatabasefolder = '/mnt/d/Finn/KPUM_NODDI/DICOM_KPUM_NODDI/sourcedata'
studydir = '/mnt/s/Research/Projects/KPUM_NODDI/Data'

# nifti base folder to go within studydir
outputniftibasefolder = 'nifti'

###################################################################################
# User input

# Participant details
subject = '0XX'
session = 'MR1'

# Define good DCM datafolder as input (and if no data is not good enough or does not exist, leave blank)
dki_dirAP = 's17_DKI_2.5mm_NODDI_dir-AP_79dir_0_1000_2000_DFC_MIX'
dki_dirAP_volb0 = '0' # Important - counting starts at 0 (i.e. first b0 volume has index 0)

dki_dirPA = 's16_DKI_2.5mm_NODDI_dir-PA_2dir_0_DFC_MIX'
dki_dirPA_volb0 = '1' # Important - counting starts at 0 (i.e. first b0 volume has index 0)

t2w_mcrib = 's05_T2W_MCRIB'

t1w_mprage = ''

###################################################################################

In [ ]:
# Create conversion file sourcedata2nifti.tsv
import os, subprocess
import pandas as pd

# Define I/O folders
sourcedatainputfolder = os.path.join(sourcedatabasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
niftioutputfolder = os.path.join(studydir, outputniftibasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
if not os.path.exists(niftioutputfolder): # then make this directory
    os.makedirs(niftioutputfolder)

# Create list for matching file transfers
sourcedatalist = [dki_dirAP,
                  dki_dirPA,
                  t2w_mcrib,
                  t1w_mprage]
niftidatalist = [os.path.join('dwi',f'sub-{subject}_ses-{session}_dir-AP_dwi.nii'),
                 os.path.join('fmap',f'sub-{subject}_ses-{session}_dir-PA_epi.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-mcrib_T2w.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-mprage_T1w.nii')]
# Make this into a DataFrame
sample = {'sourcedata_input': sourcedatalist, 
          'nifti_output': niftidatalist}
df = pd.DataFrame(sample)
# Locate only the files that are going to be transferred (i.e. not have entry '' in sourcedatalist)
df_filtered = (df.loc[df['sourcedata_input'] != ''])
# and save to tsv-file in niftioutputfolder
df_filtered.to_csv(os.path.join(niftioutputfolder, f'sub-{subject}_ses-{session}_sourcedata2nifti.tsv'), sep="\t", index=False)

# Now use this df_filtered / tsv-file for the data transfer
# and perform conversion using dcm2niix
for ind in df_filtered.index:
        in_path = os.path.join(sourcedatainputfolder, df_filtered['sourcedata_input'][ind])
        out_filebase = os.path.splitext(df_filtered['nifti_output'][ind])[0] # workaround to get rid of file-extension
        print('Start converting DCMs in', in_path,'and save in', os.path.join(niftioutputfolder, out_filebase))
        p=subprocess.Popen(f"dcm2niix -a y -d 1 -b y -w 1 -o {niftioutputfolder} -f {out_filebase} {in_path}", stdout=subprocess.PIPE, shell=True)  
        print('Finished converting DCMs in', in_path,'and save in', os.path.join(niftioutputfolder, out_filebase))
        



In [ ]:

# Now sort out the fieldmaps (fmap)

###################################################################
# DKI_dirPA - we will only keep the volume corresponding to the dki_dirPA_volb0
relativefilepath = df_filtered.query('nifti_output.str.contains("dir-PA_epi")').at[1, 'nifti_output']
absfilepath = os.path.join(niftioutputfolder, relativefilepath)
absfiledir = os.path.dirname(absfilepath)
filename = os.path.basename(absfilepath)
filebasename = os.path.splitext(filename)[0]

# build up mrtrixcommand1
jsonimport = f'-json_import ' + os.path.join(absfiledir,f'{filebasename}.json') 
fslgradimport = ''
extractvol = dki_dirPA_volb0
fileinput = absfilepath
mrtrixcommand1 = f"mrconvert -force {jsonimport} {fslgradimport} -coord 3 {extractvol} -axes 0,1,2 {fileinput} - "

# build up mrtrixcommand2
fileoutput = absfilepath
jsonexport = f'-json_export ' + os.path.join(absfiledir,f'{filebasename}.json') 
mrtrixcommand2 = f"mrconvert -force {jsonexport} - {fileoutput}"

# put commands together and perform alteration
mrtrixcommand = mrtrixcommand1 + ' | ' + mrtrixcommand2
p=subprocess.Popen(mrtrixcommand, stdout=subprocess.PIPE, shell=True)  


In [ ]:

import pandas as pd

# Now sort out the fieldmaps (fmap)
del df
df=pd.read_csv(os.path.join(niftioutputfolder, f'sub-{subject}_ses-{session}_sourcedata2nifti.tsv'), sep="\t")

###################################################################
# DKI_dirAP - we will only keep the volume corresponding to the dki_dirAP_volb0
relativefilepath = df.query('nifti_output.str.contains("dir-AP_dwi")').at[1, 'nifti_output']
absfilepath = os.path.join(niftioutputfolder, relativefilepath)
absfiledir = os.path.dirname(absfilepath)
filename = os.path.basename(absfilepath)
filebasename = os.path.splitext(filename)[0]

# build up mrtrixcommand1
jsonimport = f'-json_import ' + os.path.join(absfiledir,f'{filebasename}.json') 
fslgradimport = ''
extractvol = dki_dirPA_volb0
fileinput = absfilepath
mrtrixcommand1 = f"mrconvert -force {jsonimport} {fslgradimport} -coord 3 {extractvol} -axes 0,1,2 {fileinput} - "

# build up mrtrixcommand2
fileoutput = absfilepath
jsonexport = f'-json_export ' + os.path.join(absfiledir,f'{filebasename}.json') 
mrtrixcommand2 = f"mrconvert -force {jsonexport} - {fileoutput}"

# put commands together and perform alteration
mrtrixcommand = mrtrixcommand1 + ' | ' + mrtrixcommand2
p=subprocess.Popen(mrtrixcommand, stdout=subprocess.PIPE, shell=True)  
